#Environment Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Put Data in DataFrame (Articles)

In [2]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd

# Place dataset path here
FNNpath = "../Datasets/FNN Titles/"
CoAIDpath = "../Datasets/CoAID/"

politiFactFake = pd.read_csv(FNNpath+"politifact_fake.csv", usecols=['title'])
politiFactFake['label']=0
politiFactTrue = pd.read_csv(FNNpath+"politifact_real.csv", usecols=['title'], nrows=len(politiFactFake.values))
politiFactTrue['label']=1


dfTotal = pd.concat([politiFactTrue, politiFactFake])


X = dfTotal['title'].values
y = dfTotal['label'].values
print("len(X)", len(X))
print("len(y)", len(y))

len(X) 864
len(y) 864


# Finetune BERT on PolitiFact only

In [3]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 14.4 MB/s 
     |████████████████████████████████| 1.1 MB 63.3 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
     |████████████████████████████████| 140 kB 73.8 MB/s 
     |████████████████████████████████| 212 kB 71.5 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
     |████████████████████████████████| 271 kB 76.8 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 73.8 MB/s 
     |████████████████████████████████| 112 kB 41.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing in

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.8 MB/s 
     |████████████████████████████████| 6.6 MB 60.1 MB/s 


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer("Attention is all you need")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

{'input_ids': [101, 1335, 5208, 2116, 1110, 1155, 1128, 1444, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
from sklearn.utils import shuffle
dfTotal = shuffle(dfTotal)
from datasets import Dataset

train_dataset = Dataset.from_pandas(dfTotal)

In [6]:
def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

train_dataset = train_dataset.map(tokenize_data, batched=True)

from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

from transformers import Trainer

trainer_train = Trainer(
    model=model, args=training_args, train_dataset=train_dataset
)

Parameter 'function'=<function tokenize_data at 0x7f0a4b3ba950> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
#Train the model on 75% of dfTotal 
trainer_train.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 864
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 324


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=324, training_loss=0.24435789791154272, metrics={'train_runtime': 144.8836, 'train_samples_per_second': 17.89, 'train_steps_per_second': 2.236, 'total_flos': 681983855493120.0, 'train_loss': 0.24435789791154272, 'epoch': 3.0})

In [8]:
model_path = '../models/finetune_bert_on_PolitiFact_only.pt'
torch.save(model.state_dict(),model_path)

In [9]:
path_to_external_validation_dataset = ".../Datasets/External_Validation_Datasets//" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_to_external_validation_dataset+"External_Validation1.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0
  
dfTotal_external = dfTotal_external.dropna()
dfTotal_external['label'] = dfTotal_external['label'].apply(lambda label: binaryLabel(str(label)))
dataset_external = Dataset.from_pandas(dfTotal_external)

In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

dataset_external = dataset_external.map(tokenize_data, batched=True)

trainer_eval = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.7092248830947995,
 'eval_loss': 1.2808611392974854,
 'eval_runtime': 131.7888,
 'eval_samples_per_second': 53.548,
 'eval_steps_per_second': 6.7}